In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# Put input_docs_sample.zip in your Google Drive

!rm -rf input_docs
!cp /content/drive/My\ Drive/input_docs.zip .
# !cp /content/drive/My\ Drive/input_docs_sample.zip .
!unzip input_docs.zip > /dev/null
# !unzip input_docs_sample.zip > /dev/null
!ls input_docs/ | wc -l

# for the real collection change above input_docs_sample.zip to input_docs.zip
# for the sample collection of 5 docs, the process is fast
# for the real collection, the process takes about 6 min (start to finish, the whole notebook) 

19026


In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

**Create an RDD from a text file**

Each line of the text file becomes an element of the RDD.

In [7]:
%%time

# wholeTextFiles generates an RDD of pair values, 
# where the key is the full path of each file, the value is the content of each file
#input = sc.wholeTextFiles("/content/drive/My\ Drive/input_docs");
input = sc.wholeTextFiles("input_docs");

# Now we strip the prefix of filenames and leave only the basename. 
# e.g. 'file:/content/drive/My Drive/Colab Notebooks/data_spark/input_docs/3.html'
# becomes '3.html' 
import os


#(did,text)
input2 = input.map(lambda x: (int(os.path.basename(x[0]).split(".")[0]), x[1]))

print(input2.take(1))

[(12959, "<H2> 3-APR-1987 11:55:12.42</H2>\r\n<H2>POLYSAR IN JOINT VENTURE ON FINLAND LATEX PLANT</H2>\r\nPolysar Ltd, wholly owned by\nCanada Development Corp, said it agreed to form a joint\nventure with Raision Tehtaat, of Finland, to build a synthetic\nrubber latex plant in southern Finland.\n    Project cost and plant capacity were not disclosed.\n    The joint venture, to be 51 pct owned by Raision Tehtaat\nand 49 pct by Polysar, will build a plant at Anjalankoski, east\nof Helsinki, to produce carboxylated styrene-butadiene latex.\n    The plant's production will be used by Finland's paper\nindustry for making coated paper and paper board products.\n \n ")]
CPU times: user 40.4 ms, sys: 7.38 ms, total: 47.8 ms
Wall time: 3min 3s


In [8]:
%%time

# Doc to wordlist function
# The output will be a list of tuples such as 
# ("apple", (3,10,10/20)), 
# where 3 is docid, 
# 10 is frequency of "apple" in this doc, 
# 20 is maxf in in this doc.

from bs4 import BeautifulSoup
from collections import Counter

import string

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

# for a given doc return a list of tuples of the form (w, (docid, freq, freq/maxfreq))
def dw(docid, htmltext):
  cleantext = BeautifulSoup(htmltext).get_text()
 
#  1.to_lower_case

  cleantext = cleantext.lower()

# 2.filtering noise from data (punctuations + digits + alphanumerics)

  # noise = string.punctuation+string.digits
  noise = '!"#$%&\()*+,-:;<=>?@[\\]^_`{|}~'+string.digits
  translation = str.maketrans(string.ascii_letters, string.ascii_letters, noise)
  cleantext = cleantext.translate(translation)  

  trans1 = str.maketrans('\n\r\t\'/.','      ')
  cleantext = cleantext.translate(trans1)

# 3. stop words 
  cleantext_sw = [word for word in cleantext.split(' ') if word not in stop_words]
  cleantext = ''


# 4.stemming/lemmatization

  for word in cleantext_sw:
  #  temp = ps.stem(word)
   temp = lem.lemmatize(word)
  #  temp = ps.stem(temp)
   cleantext = cleantext+temp+' '
  
# 5. list of distinct words

  cleantext_distinct = list(dict.fromkeys(cleantext.split(' ')))
  cleantext_distinct.sort()

  try:
    cleantext_distinct_2 = [word for word in cleantext_distinct if len(word)>1]
    # cleantext_distinct_2.remove('')
  except:
    print('no strings< length 2')

# count of occurrences of distinct words in doc
  count_list = []
  list_tf = []
  for word in cleantext_distinct_2:
    count_list.append(cleantext.split(' ').count(word))

  max_count_list = max(count_list)

  for word in cleantext_distinct_2:
    list_tf.append((word,(docid,cleantext.split(' ').count(word),(cleantext.split(' ').count(word)/(max_count_list)))))

  #TODO
  #returning just a dummy result
  # return [(_,(_,_,cleantext_distinct_2))]
  return list_tf

word_docid_freq_tf = input2.flatMap(lambda x: dw(x[0],x[1]))


# print(word_docid_freq_tf.take(1))
print(word_docid_freq_tf.take(435))
# print(word_docid_freq_tf.take(2)[0][0])
# print(word_docid_freq_tf.take(2)[1][1])

# print(word_docid_freq_tf.count())

[('agreed', (12959, 1, 0.2)), ('anjalankoski', (12959, 1, 0.2)), ('apr', (12959, 1, 0.2)), ('board', (12959, 1, 0.2)), ('build', (12959, 2, 0.4)), ('canada', (12959, 1, 0.2)), ('capacity', (12959, 1, 0.2)), ('carboxylated', (12959, 1, 0.2)), ('coated', (12959, 1, 0.2)), ('corp', (12959, 1, 0.2)), ('cost', (12959, 1, 0.2)), ('development', (12959, 1, 0.2)), ('disclosed', (12959, 1, 0.2)), ('east', (12959, 1, 0.2)), ('finland', (12959, 4, 0.8)), ('form', (12959, 1, 0.2)), ('helsinki', (12959, 1, 0.2)), ('industry', (12959, 1, 0.2)), ('joint', (12959, 3, 0.6)), ('latex', (12959, 3, 0.6)), ('ltd', (12959, 1, 0.2)), ('making', (12959, 1, 0.2)), ('owned', (12959, 2, 0.4)), ('paper', (12959, 3, 0.6)), ('pct', (12959, 2, 0.4)), ('plant', (12959, 5, 1.0)), ('polysar', (12959, 3, 0.6)), ('produce', (12959, 1, 0.2)), ('product', (12959, 1, 0.2)), ('production', (12959, 1, 0.2)), ('project', (12959, 1, 0.2)), ('raision', (12959, 2, 0.4)), ('rubber', (12959, 1, 0.2)), ('said', (12959, 1, 0.2)), ('s

Expected output (all expected results are on the small sample):

<pre>
[('feb', (1, 1, 0.07142857142857142)), ('bahia', (1, 5, 0.35714285714285715))]
</pre>

In [9]:
%%time
# Now create an RDD as follows 
# (word, [(did1,freq1,tf1), (did2,freq2,tf2), ...])

# # 1. converting rdd to list
# rdd_to_list = word_docid_freq_tf.collect()


# # 2. list of keys from rdd
# list_keys = []
# for el in rdd_to_list:
#   list_keys.append(el[0])


# # 3. list of distinct keys
# list_keys_d = list(dict.fromkeys(list_keys))


# list_to_rdd = [] 

# # 4. indices of distinct keys in list_keys

# for el in list_keys_d:
#   ind = [index for index, value in enumerate(list_keys) if value == el]
#   sub_list = []

#   for i in ind:
#     sub_list.append(rdd_to_list[i][1])
#   list_to_rdd.append((el,sub_list))


word_postinglist_freq_tf =  word_docid_freq_tf.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x ,y: x+y)

#TODO
# creating a dummy RDD
# word_postinglist_freq_tf = sc.parallelize([ ('test', [(1, 1, 0.5), (2, 1, 0.2)]) ])
# word_postinglist_freq_tf = sc.parallelize(tmp)


CPU times: user 14.7 ms, sys: 2.72 ms, total: 17.4 ms
Wall time: 52.5 ms


In [0]:
# # print(word_postinglist_freq_tf.map(lambda x : (x[0], list(x[1]))).take(1))
# print(word_postinglist_freq_tf.take(1))

Expected output

<pre>
[('feb', [(1, 1, 0.07142857142857142), (2, 1, 0.2), (5, 1, 0.16666666666666666), (3, 1, 0.3333333333333333), (4, 1, 0.07142857142857142)]), ('bahia', [(1, 5, 0.35714285714285715)])]
</pre>

In [11]:
%%time

# (word, [(did,freq,tfidf), ...])
# We easily obtain idf as 1/len(postinglist_tf)
# idf = 1/len(postinglist_tf)

rdd_list_tfidf = word_postinglist_freq_tf.collect()

# print(rdd_list_tfidf)


for pair in rdd_list_tfidf:
  # pair[0] #word
  # pair[1] #list of tuples of word
  s_list = []
  
  for tup in pair[1]:
    tf = tup[2] #tf of word in tuple
    tf_idf = tf/len(pair[1])
    s_list.append(tup[:2]+(tf_idf,))
  rdd_list_tfidf[rdd_list_tfidf.index(pair)] = pair[:1]+(s_list,)

l = []
for el in rdd_list_tfidf:
  temp_list = el[1]
  sorted_list = sorted(temp_list, key=lambda x: x[0])
  t = el[:1]+(sorted_list,)
  l.append(t)



#TODO

# creating a dummy RDD
# word_postinglist_freq_tfidf = sc.parallelize([('test', [(1,3,0.12), (2,5,0.876)])])
word_postinglist_freq_tfidf = sc.parallelize(l)

# print(word_postinglist_freq_tfidf.take(100))

CPU times: user 30.6 s, sys: 173 ms, total: 30.8 s
Wall time: 4min 42s


Expected output

<pre>
[('feb', [(1, 1, 0.014285714285714285), (2, 1, 0.04), (5, 1, 0.03333333333333333), (3, 1, 0.06666666666666667), (4, 1, 0.014285714285714285)])]
</pre>

In [12]:
%%time

# Now, we would like to obtain the magnitude of each doc.
# First, produce (did, (freq,tfidf)) for each word of doc did; 
# We do don't need the word itself, just its (freq,tfidf). 
# Then, do reduceByKey on these tuples and obtain maxfreq and 
# magnitude (squared) for each document. 

import math

#TODO

rdd_list_mag = word_postinglist_freq_tfidf.collect()

mag_list_rdd = []

for el in rdd_list_mag:
  for e in el[1]:
    did = e[0]
    freq = e[1]
    tf_idf = e[2]**2
    x = (did,(freq,tf_idf))
    mag_list_rdd.append(x)


# RDD of (did,(freq,tfidf)) tuples
# creating a dummy RDD
# did_freq_tfidfsq_rdd = sc.parallelize([(1,(3,0.12)), (1,(5,0.13)), (2,(2,0.11)), (2,(6,0.14))])
did_freq_tfidfsq_rdd = sc.parallelize(mag_list_rdd)

print(did_freq_tfidfsq_rdd.take(500))


temp =  did_freq_tfidfsq_rdd.reduceByKey(lambda x,y: (max(x[0],y[0]),(x[1]+y[1])))
t = temp.collect()
# Produce (did,(maxf,magnitudesq))
# creating a dummy RDD
# doc_maxf_mag = sc.parallelize([[(1, (5, 0.23)), (2, (6, 0.34))]])
doc_maxf_mag = sc.parallelize(t)

print(doc_maxf_mag.take(5))
print(doc_maxf_mag.count())


[(12959, (1, 0.04000000000000001)), (6, (7, 1.2888522264614858e-08)), (2206, (2, 7.1123600415344045e-09)), (2239, (1, 4.939138917732226e-09)), (4305, (1, 1.1113062564897507e-08)), (10367, (1, 5.487932130813584e-10)), (10758, (1, 2.778265641224377e-09)), (11367, (2, 7.1123600415344045e-09)), (11773, (1, 2.2679719520198993e-10)), (11774, (1, 2.778265641224377e-09)), (11775, (1, 1.7780900103836011e-09)), (11776, (1, 3.0869618235826413e-10)), (11777, (1, 1.2347847294330565e-09)), (11778, (1, 1.7780900103836011e-09)), (11779, (1, 1.2347847294330565e-09)), (11780, (1, 1.7780900103836011e-09)), (11781, (1, 2.2679719520198993e-10)), (11782, (1, 1.7364160257652355e-10)), (11783, (1, 4.939138917732226e-09)), (11784, (1, 6.945664103060942e-10)), (11785, (1, 4.939138917732226e-09)), (11786, (1, 1.7780900103836011e-09)), (11787, (1, 3.6737396908752093e-10)), (11788, (1, 1.2347847294330565e-09)), (11789, (1, 1.7780900103836011e-09)), (11791, (1, 1.7780900103836011e-09)), (11792, (1, 2.77826564122437

Excpected result

<pre>
[(1, (1, 0.0002040816326530612)), (2, (1, 0.0016))]
[(2, (5, 3.894100000000001)), (4, (14, 2.94553429705215))]
</pre>

In [0]:
!rm -rf inv_idx
word_postinglist_freq_tfidf.saveAsTextFile("inv_idx");

In [0]:
!rm -rf doc_mag
doc_maxf_mag.saveAsTextFile("doc_mag");

In [15]:
!ls -lrt inv_idx
!head inv_idx/part-00001
!wc -l inv_idx/part-00000
!wc -l inv_idx/part-00001
!cat inv_idx/part-00000 inv_idx/part-00001 > /content/drive/My\ Drive/inv_idx.txt
!wc -l /content/drive/My\ Drive/inv_idx.txt

total 36192
-rw-r--r-- 1 root root 16857212 Apr  3 05:14 part-00001
-rw-r--r-- 1 root root 20198778 Apr  3 05:14 part-00000
-rw-r--r-- 1 root root        0 Apr  3 05:14 _SUCCESS
('structurally', [(18422, 1, 0.09090909090909091)])
('cainsloan', [(16332, 3, 0.21428571428571427)])
('terrific', [(16332, 1, 0.07142857142857142)])
('compris', [(11311, 4, 0.8)])
('singed', [(11311, 1, 0.2)])
('florestais', [(11854, 1, 0.25)])
('portobased', [(11854, 1, 0.25)])
('siaf', [(11854, 2, 0.5)])
('cfsgw', [(2331, 1, 0.125)])
('copelco', [(2331, 3, 0.375)])
21504 inv_idx/part-00000
22032 inv_idx/part-00001
43536 /content/drive/My Drive/inv_idx.txt


In [16]:
!ls -lrt doc_mag
!head doc_mag/part-00000
!wc -l doc_mag/part-00000
!wc -l doc_mag/part-00001
!cat doc_mag/part-00000 doc_mag/part-00001 > /content/drive/My\ Drive/doc_mag.txt
!wc -l /content/drive/My\ Drive/doc_mag.txt

total 636
-rw-r--r-- 1 root root 333803 Apr  3 05:14 part-00001
-rw-r--r-- 1 root root 313217 Apr  3 05:14 part-00000
-rw-r--r-- 1 root root      0 Apr  3 05:14 _SUCCESS
(6, (13, 0.01798154113965737))
(2206, (5, 0.04024900366924768))
(10758, (4, 0.002466342786161299))
(11774, (4, 0.0002555244853909414))
(11776, (12, 0.03825746612795345))
(11778, (5, 0.05386814819153642))
(11780, (5, 0.24170615040358373))
(11782, (16, 0.012423480182132962))
(11784, (8, 0.006635539213534009))
(11786, (5, 0.13027503165110568))
9216 doc_mag/part-00000
9810 doc_mag/part-00001
19026 /content/drive/My Drive/doc_mag.txt
